### Modules importeren

In [4]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

import os
from loguru import logger # pip install loguru
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

servername = 'LAPTOP-NBUM6TMN' # PAS DIT AAN NAAR JE EIGEN SERVER!!!

# Bron-databases inlezen

### Connecties maken

In [5]:
# Functie om een connectie te maken met een SQL Server database
def establish_connection(servername, database):
    logger.info(f'Establishing connection with {database} on {servername}...')
    try:
        conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + servername + 
                        ';DATABASE=' + database + ';Trusted_Connection=yes')
        logger.success(f'Connection established with {database} on {servername}.')
        return conn
    except Exception as e:
        logger.error(f'Failed to establish connection with {database} on {servername}. Error: {e}')
        return None

# Functie om een connectie te maken met een Microsoft Access database
def establish_access_connection(database):
    db_name = os.path.basename(database)
    logger.info(f'Establishing connection with {db_name}...')
    try:
        conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + database)
        logger.success(f'Connection established with {db_name}.')
        return conn
    except Exception as e:
        logger.error(f'Failed to establish connection with {db_name}. Error: {e}')
        return None

# Connecties maken met de bronnen
northwind_ssms_conn = establish_connection(servername, 'Northwind')
northwind_ssms_cursor = northwind_ssms_conn.cursor()

adventureworks_ssms_conn = establish_connection(servername, 'AdventureWorks2019')
adventureworks_ssms_cursor = adventureworks_ssms_conn.cursor()

aenc_access_conn = establish_access_connection('../data/raw/aenc.accdb')
aenc_access_cursor = aenc_access_conn.cursor()

# SQLite, maak nieuwe databases aan als deze nog niet bestaat
try:
    logger.info('Establishing connection with SQLite databases...')
    northwind_sqlite_conn = sqlite3.connect('../data/processed/northwind.sqlite')
    adventureworks_sqlite_conn = sqlite3.connect('../data/processed/adventureworks.sqlite')
    aenc_sqlite_conn = sqlite3.connect('../data/processed/aenc.sqlite')
    logger.success('Connection established with SQLite databases.')
except Exception as e:
    logger.error(f'Failed to establish connection with SQLite databases. Error: {e}')

2024-05-23 10:33:17.118 | INFO     | __main__:establish_connection:3 - Establishing connection with Northwind on LAPTOP-NBUM6TMN...


2024-05-23 10:33:21.368 | SUCCESS  | __main__:establish_connection:7 - Connection established with Northwind on LAPTOP-NBUM6TMN.
2024-05-23 10:33:21.368 | INFO     | __main__:establish_connection:3 - Establishing connection with AdventureWorks2019 on LAPTOP-NBUM6TMN...
2024-05-23 10:33:25.528 | SUCCESS  | __main__:establish_connection:7 - Connection established with AdventureWorks2019 on LAPTOP-NBUM6TMN.
2024-05-23 10:33:25.534 | INFO     | __main__:establish_access_connection:16 - Establishing connection with aenc.accdb...
2024-05-23 10:33:27.084 | SUCCESS  | __main__:establish_access_connection:19 - Connection established with aenc.accdb.
2024-05-23 10:33:27.088 | INFO     | __main__:<module>:37 - Establishing connection with SQLite databases...
2024-05-23 10:33:27.108 | SUCCESS  | __main__:<module>:41 - Connection established with SQLite databases.


### Bron-databases overzetten naar SQLite bestanden

In [34]:
# Functie om tabellen van een bron-database over te zetten naar een SQLite database
def transfer_tables_to_sqlite(tables, conn, sqlite_conn):
    db_name = sqlite_conn.execute("PRAGMA database_list;").fetchall()[0][2] # Padlocatie van de database
    db_name = os.path.basename(db_name) # Alleen de naam van de database
    logger.info(f'Transferring tables to SQLite database {db_name}...')

    failed_tables = []

    for table in tables:
        try:
            sql_query = f'SELECT * FROM {table};'
            data = pd.read_sql(sql_query, conn)
            sqlite_table_name = table.replace('[', '').replace(']', '')  # Verwijder vierkante haken voor tabellen met een spatie
            data.to_sql(sqlite_table_name, sqlite_conn, index=False, if_exists='replace')
        except Exception as e:
            logger.error(f'Error transferring table {table} to SQLite database {db_name}. Error: {e}')
            failed_tables.append(table)
            break
    
    if len(failed_tables) > 0:
        logger.warning(f'Tables that failed to transfer: {failed_tables}. Rest of the tables were transferred successfully to SQLite database {db_name}.')
    else:
        logger.success(f'Transferred tables to SQLite database {db_name}!')


# Tabellen die worden overgezet naar SQLite
northwind_tables = [
    'Categories', 
    'CustomerCustomerDemo', 
    'CustomerDemographics', 
    'Customers', 
    'Employees', 
    'EmployeeTerritories', 
    '[Order Details]', # Vanwege de spatie in de tabelnaam moet deze tussen vierkante haken (I <3 PYTHON!!!)
    'Orders', 
    'Products', 
    'Region', 
    'Shippers', 
    'Suppliers', 
    'Territories'
    ]

adventureworks_tables = [
    'dbo.AWBuildVersion',
    'dbo.DatabaseLog',
    'dbo.ErrorLog',
    'HumanResources.Department',
    'HumanResources.Employee',
    'HumanResources.EmployeeDepartmentHistory',
    'HumanResources.EmployeePayHistory',
    'HumanResources.JobCandidate',
    'HumanResources.Shift',
    'Person.Address',
    'Person.AddressType',
    'Person.BusinessEntity',
    'Person.BusinessEntityAddress',
    'Person.BusinessEntityContact',
    'Person.ContactType',
    'Person.CountryRegion',
    'Person.EmailAddress',
    'Person.Password',
    'Person.Person',
    'Person.PersonPhone',
    'Person.PhoneNumberType',
    'Person.StateProvince',
    'Production.BillOfMaterials',
    'Production.Culture',
    'Production.Document',
    'Production.Illustration',
    'Production.Location',
    'Production.Product',
    'Production.ProductCategory',
    'Production.ProductCostHistory',
    'Production.ProductDescription',
    'Production.ProductDocument',
    'Production.ProductInventory',
    'Production.ProductListPriceHistory',
    'Production.ProductModel',
    'Production.ProductModelIllustration',
    'Production.ProductModelProductDescriptionCulture',
    'Production.ProductPhoto',
    'Production.ProductProductPhoto',
    'Production.ProductReview',
    'Production.ProductSubcategory',
    'Production.ScrapReason',
    'Production.TransactionHistory',
    'Production.TransactionHistoryArchive',
    'Production.UnitMeasure',
    'Production.WorkOrder',
    'Production.WorkOrderRouting',
    'Purchasing.ProductVendor',
    'Purchasing.PurchaseOrderDetail',
    'Purchasing.PurchaseOrderHeader',
    'Purchasing.ShipMethod',
    'Purchasing.Vendor',
    'Sales.CountryRegionCurrency',
    'Sales.CreditCard',
    'Sales.Currency',
    'Sales.CurrencyRate',
    'Sales.Customer',
    'Sales.PersonCreditCard',
    'Sales.SalesOrderDetail',
    'Sales.SalesOrderHeader',
    'Sales.SalesOrderHeaderSalesReason',
    'Sales.SalesPerson',
    'Sales.SalesPersonQuotaHistory',
    'Sales.SalesReason',
    'Sales.SalesTaxRate',
    'Sales.SalesTerritory',
    'Sales.SalesTerritoryHistory',
    'Sales.ShoppingCartItem',
    'Sales.SpecialOffer',
    'Sales.SpecialOfferProduct',
    'Sales.Store'
]

aenc_tables = [
    'bonus',
    'customer',
    'department',
    'employee',
    'product',
    'region',
    'sales_order',
    'sales_order_item',
    'state'
]

transfer_tables_to_sqlite(northwind_tables, northwind_ssms_conn, northwind_sqlite_conn)
transfer_tables_to_sqlite(adventureworks_tables, adventureworks_ssms_conn, adventureworks_sqlite_conn)
transfer_tables_to_sqlite(aenc_tables, aenc_access_conn, aenc_sqlite_conn)

2024-05-20 20:25:17.176 | INFO     | __main__:transfer_tables_to_sqlite:5 - Transferring tables to SQLite database northwind.sqlite...
2024-05-20 20:25:21.233 | SUCCESS  | __main__:transfer_tables_to_sqlite:23 - Transferred tables to SQLite database northwind.sqlite!
2024-05-20 20:25:21.234 | INFO     | __main__:transfer_tables_to_sqlite:5 - Transferring tables to SQLite database adventureworks.sqlite...
2024-05-20 20:26:08.719 | SUCCESS  | __main__:transfer_tables_to_sqlite:23 - Transferred tables to SQLite database adventureworks.sqlite!
2024-05-20 20:26:08.720 | INFO     | __main__:transfer_tables_to_sqlite:5 - Transferring tables to SQLite database aenc.sqlite...
2024-05-20 20:26:11.729 | SUCCESS  | __main__:transfer_tables_to_sqlite:23 - Transferred tables to SQLite database aenc.sqlite!


# Data overzetten naar de Data-Warehouse

### Order_Details

In [35]:
def transfer_data_to_orderdetails(conn):

    try:
        logger.info('Wiping old data in Order_Details...')
        conn.execute('DELETE FROM Order_Details;')
        conn.commit()
        logger.success('Wiped old data in Order_Details.')
    except Exception as e:
        logger.error(f'Error wiping old data in Order_Details. Error: {e}')
        return
    
    try:
        logger.info('Connecting to the order data sources...')
        aw_sales_order_header = pd.read_sql('SELECT * FROM "Sales.SalesOrderHeader"', adventureworks_sqlite_conn)
        aenc_sales_order = pd.read_sql('SELECT * FROM sales_order', aenc_sqlite_conn)
        northwind_sales_order = pd.read_sql('SELECT * FROM Orders', northwind_sqlite_conn)
        logger.success('Connected to the order data sources.')
    except Exception as e:
        logger.error(f'Error connecting to the order data sources. Error: {e}')
        return

    surrogate_key = 1

    try:

        logger.info('Transferring the order data from Northwind to Order_Details...')

        for index, row in northwind_sales_order.iterrows():
            
            order_details_ids = pd.read_sql(f'SELECT * FROM "Order Details" WHERE OrderID = {row["OrderID"]}', northwind_sqlite_conn)

            customer_id = row['CustomerID']
            employee_id = row['EmployeeID']
            order_date = row['OrderDate']
            required_date = row['RequiredDate']
            shipped_date = row['ShippedDate']
            ship_via = row['ShipVia']
            freight = row['Freight']
            ship_name = row['ShipName']
            ship_address = row['ShipAddress']
            ship_city = row['ShipCity']
            ship_region = row['ShipRegion']
            ship_postal_code = row['ShipPostalCode']
            ship_country = row['ShipCountry']

            for index, row in order_details_ids.iterrows():

                query = """
                    INSERT INTO Order_Details (
                        ID,
                        CustomerID,
                        EmployeeID,
                        OrderDate,
                        RequiredDate,
                        ShippedDate,
                        ShipVia,
                        Freight,
                        ShipName,
                        ShipAddress,
                        ShipCity,
                        ShipRegion,
                        ShipPostalCode,
                        ShipCounty,
                        ProductID,
                        UnitPrice,
                        Quantity,
                        Discount,
                        SK
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                """

                params = (
                    row['OrderID'],
                    customer_id,
                    employee_id,
                    order_date,
                    required_date,
                    shipped_date,
                    ship_via,
                    freight,
                    ship_name,
                    ship_address,
                    ship_city,
                    ship_region,
                    ship_postal_code,
                    ship_country,
                    row['ProductID'],
                    row['UnitPrice'],
                    row['Quantity'],
                    row['Discount'],
                    surrogate_key
                )

                conn.execute(query, params)
                conn.commit()

                surrogate_key += 1
        
        logger.success('Transferred the order data from Northwind to Order_Details.')
    except Exception as e:
        logger.error(f'Error transferring the order data from Northwind to Order_Details. Error: {e}')
        return


    try:
        logger.info('Transferring the order data from AENC to Order_Details...')

        for index, row in aenc_sales_order.iterrows():

            sales_order_item_ids = pd.read_sql(f'SELECT * FROM sales_order_item WHERE id = {row["id"]}', aenc_sqlite_conn)

            customer_id = row['cust_id']
            order_date = row['order_date']
            region = row['region']
            sales_person_id = row['sales_rep']

            for index, row in sales_order_item_ids.iterrows():
                query = """
                    INSERT INTO Order_Details (
                        ID,
                        CustomerID,
                        OrderDate,
                        region,
                        SalesPersonID,
                        line_id,
                        ProductID,
                        Quantity,
                        ShipDate,
                        SK
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                """

                params = (
                    row['id'],
                    customer_id,
                    order_date,
                    region,
                    sales_person_id,
                    row['line_id'],
                    row['prod_id'],
                    row['quantity'],
                    row['ship_date'],
                    surrogate_key
                )
        
                conn.execute(query, params)
                conn.commit()

                surrogate_key += 1

        logger.success('Transferred the order data from AENC to Order_Details.')
    except Exception as e:
        logger.error(f'Error transferring the order data from AENC to Order_Details. Error: {e}')
        return

    try:
        logger.info('Transferring the order data from AdventureWorks to Order_Details...')

        for index, row in aw_sales_order_header.iterrows():

            sales_reason_id = pd.read_sql(f'SELECT * FROM "Sales.SalesOrderHeaderSalesReason" WHERE SalesOrderID = {row["SalesOrderID"]} LIMIT 1', adventureworks_sqlite_conn)
            sales_order_detail_ids = pd.read_sql(f'SELECT * FROM "Sales.SalesOrderDetail" WHERE SalesOrderID = {row["SalesOrderID"]}', adventureworks_sqlite_conn)
            freight = row['Freight']
            ship_method_id = row['ShipMethodID']
            ship_method_name = pd.read_sql(f'SELECT Name from "Purchasing.ShipMethod" WHERE ShipMethodID = {ship_method_id}', adventureworks_sqlite_conn)['Name'].values[0]
            ship_method_base = pd.read_sql(f'SELECT ShipBase from "Purchasing.ShipMethod" WHERE ShipMethodID = {ship_method_id}', adventureworks_sqlite_conn)['ShipBase'].values[0]
            ship_method_rate = pd.read_sql(f'SELECT ShipRate from "Purchasing.ShipMethod" WHERE ShipMethodID = {ship_method_id}', adventureworks_sqlite_conn)['ShipRate'].values[0]
            due_date = row['DueDate']
            online_order_flag = row['OnlineOrderFlag']
            order_date = row['OrderDate']
            ship_date = row['ShipDate']
            status = row['Status']
            sub_total = row['SubTotal']
            tax_amt = row['TaxAmt']
            total_due = row['TotalDue']
            customer_id = row['CustomerID']
            bill_to_address_id = row['BillToAddressID']
            credit_card_id = row['CreditCardID']
            currency_rate_id = row['CurrencyRateID']
            sales_person_id = row['SalesPersonID']
            ship_to_address_id = row['ShipToAddressID']
            territory_id = row['TerritoryID']

            for index, row in sales_order_detail_ids.iterrows():
                query = """
                    INSERT INTO Order_Details (
                        ID, 
                        SalesReasonID, 
                        SalesOrderDetailID,
                        UnitPrice,
                        Quantity,
                        Discount,
                        Freight,
                        ShipName,
                        ShipBase,
                        ShipRate,
                        DueDate,
                        OnlineOrderFlag,
                        OrderDate,
                        ShipDate,
                        Status,
                        SubTotal,
                        TaxAmt,
                        TotalDue,
                        CustomerID,
                        ProductID,
                        BillToAdressID,
                        CreditCardID,
                        CurrencyRateID,
                        SalesPersonID,
                        ShipMethodID,
                        ShipToAdressID,
                        TerritoryID,
                        SpecialOfferID,
                        SK
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                """

                params = (
                    row['SalesOrderID'],
                    int(sales_reason_id['SalesReasonID'].values[0]) if not sales_reason_id.empty else np.nan,
                    row['SalesOrderDetailID'],
                    row['UnitPrice'],
                    row['OrderQty'],
                    row['UnitPriceDiscount'],
                    freight,
                    ship_method_name,
                    ship_method_base,
                    ship_method_rate,
                    due_date,
                    online_order_flag,
                    order_date,
                    ship_date,
                    status,
                    sub_total,
                    tax_amt,
                    total_due,
                    customer_id,
                    row['ProductID'],
                    bill_to_address_id,
                    credit_card_id,
                    currency_rate_id,
                    sales_person_id,
                    ship_method_id,
                    ship_to_address_id,
                    territory_id,
                    row['SpecialOfferID'],
                    surrogate_key
                )

                conn.execute(query, params)
                conn.commit()

                surrogate_key += 1
    except Exception as e:
        logger.error(f'Error transferring the order data from AdventureWorks to Order_Details. Error: {e}')
        return
    
def create_empty_order_details_dataframe():
    dtypes = {
        'ID': np.int32,
        'SalesReasonID': np.int32,
        'SalesOrderDetailID': np.int32,
        'UnitPrice': np.float64,
        'Quantity': np.int32,
        'Discount': np.float64,
        'ShipVia': np.int32,
        'Freight': np.float64,
        'ShipName': 'object',
        'ShipBase': np.float64,
        'ShipRate': np.float64,
        'ShipAddress': 'object',
        'ShipCity': 'object',
        'ShipRegion': 'object',
        'ShipPostalCode': 'object',
        'ShipCounty': 'object',
        'DueDate': 'datetime64[ns]',
        'ModifiedDate': 'datetime64[ns]',
        'OnlineOrderFlag': np.bool_,
        'OrderDate': 'datetime64[ns]',
        'ShipDate': 'datetime64[ns]',
        'Status': 'object',
        'SubTotal': np.float64,
        'TaxAmt': np.float64,
        'TotalDue': np.float64,
        'RequiredDate': 'datetime64[ns]',
        'ShippedDate': 'datetime64[ns]',
        'region': 'object',
        'line_id': np.int32,
        'CustomerID': np.int32,
        'EmployeeID': np.int32,
        'ProductID': np.int32,
        'BillToAdressID': np.int32,
        'CreditCardID': np.int32,
        'CurrencyRateID': np.int32,
        'SalesPersonID': np.int32,
        'ShipMethodID': np.int32,
        'ShipToAdressID': np.int32,
        'TerritoryID': np.int32,
        'SpecialOfferID': np.int32,
        'SK': np.int32
    }

    df = pd.DataFrame(columns=dtypes.keys()).astype(dtypes)
    return df

test_database = sqlite3.connect('../data/processed/test.sqlite')
create_empty_order_details_dataframe().to_sql('Order_Details', test_database, index=False, if_exists='replace')

transfer_data_to_orderdetails(test_database)

2024-05-20 20:32:58.024 | INFO     | __main__:transfer_data_to_orderdetails:4 - Wiping old data in Order_Details...
2024-05-20 20:32:58.131 | SUCCESS  | __main__:transfer_data_to_orderdetails:7 - Wiped old data in Order_Details.
2024-05-20 20:32:58.132 | INFO     | __main__:transfer_data_to_orderdetails:13 - Connecting to the order data sources...
2024-05-20 20:32:58.485 | SUCCESS  | __main__:transfer_data_to_orderdetails:17 - Connected to the order data sources.
2024-05-20 20:32:58.485 | INFO     | __main__:transfer_data_to_orderdetails:26 - Transferring the order data from Northwind to Order_Details...
2024-05-20 20:37:14.896 | SUCCESS  | __main__:transfer_data_to_orderdetails:99 - Transferred the order data from Northwind to Order_Details.
2024-05-20 20:37:14.897 | INFO     | __main__:transfer_data_to_orderdetails:106 - Transferring the order data from AENC to Order_Details...
2024-05-20 20:39:25.240 | SUCCESS  | __main__:transfer_data_to_orderdetails:151 - Transferred the order dat

### SalesPersonQuotaHistory

In [4]:
def transfer_data_to_salespersonquotahistory(conn):

    try:
        logger.info('Wiping old data in SalesPersonQuotaHistory...')
        conn.execute('DELETE FROM SalesPersonQuotaHistory;')
        conn.commit()
        logger.success('Wiped old data in SalesPersonQuotaHistory.')
    except Exception as e:
        logger.error(f'Error wiping old data in SalesPersonQuotaHistory. Error: {e}')
        return
    
    try:
        logger.info('Connecting to the SalesPersonQuotaHistory data sources...')
        aw_sales_person_quota_history = pd.read_sql('SELECT * FROM "Sales.SalesPersonQuotaHistory"', adventureworks_sqlite_conn)
        logger.success('Connected to the SalesPersonQuotaHistory data sources.')
    except Exception as e:
        logger.error(f'Error connecting to the SalesPersonQuotaHistory data sources. Error: {e}')
        return

    surrogate_key = 1

    try:
        logger.info('Transferring the SalesPersonQuotaHistory data from AdventureWorks to SalesPersonQuotaHistory...')

        for index, row in aw_sales_person_quota_history.iterrows():
            
            query = """
                INSERT INTO SalesPersonQuotaHistory (
                    BusinessEntityID,
                    QuotaDate,
                    SalesQuota,
                    ModifiedDate,
                    SK
                ) VALUES (?, ?, ?, ?, ?);
            """

            params = (
                row['BusinessEntityID'],
                row['QuotaDate'],
                row['SalesQuota'],
                row['ModifiedDate'],
                surrogate_key
            )

            conn.execute(query, params)
            conn.commit()

            surrogate_key += 1

        logger.success('Transferred the SalesPersonQuotaHistory data from AdventureWorks to SalesPersonQuotaHistory.')

    except Exception as e:
        logger.error(f'Error transferring the SalesPersonQuotaHistory data from AdventureWorks to SalesPersonQuotaHistory. Error: {e}')
        return

def create_empty_salespersonquotahistory_dataframe():
    dtypes = {
        'BusinessEntityID': np.int32,
        'QuotaDate': 'datetime64[ns]',
        'SalesQuota': np.int32,
        'ModifiedDate': 'datetime64[ns]',
        'SK': np.int32
    }

    df = pd.DataFrame(columns=dtypes.keys()).astype(dtypes)
    return df

test_database = sqlite3.connect('../data/processed/test.sqlite')
create_empty_salespersonquotahistory_dataframe().to_sql('SalesPersonQuotaHistory', test_database, index=False, if_exists='replace')

transfer_data_to_salespersonquotahistory(test_database)

2024-05-21 18:04:29.798 | INFO     | __main__:transfer_data_to_salespersonquotahistory:4 - Wiping old data in SalesPersonQuotaHistory...
2024-05-21 18:04:29.888 | SUCCESS  | __main__:transfer_data_to_salespersonquotahistory:7 - Wiped old data in SalesPersonQuotaHistory.
2024-05-21 18:04:29.888 | INFO     | __main__:transfer_data_to_salespersonquotahistory:13 - Connecting to the SalesPersonQuotaHistory data sources...
2024-05-21 18:04:29.890 | SUCCESS  | __main__:transfer_data_to_salespersonquotahistory:15 - Connected to the SalesPersonQuotaHistory data sources.
2024-05-21 18:04:29.891 | INFO     | __main__:transfer_data_to_salespersonquotahistory:23 - Transferring the SalesPersonQuotaHistory data from AdventureWorks to SalesPersonQuotaHistory...
2024-05-21 18:04:44.084 | SUCCESS  | __main__:transfer_data_to_salespersonquotahistory:50 - Transferred the SalesPersonQuotaHistory data from AdventureWorks to SalesPersonQuotaHistory.


### SalesTerritoryHistory

In [5]:
def transfer_data_to_salesterritoryhistory(conn):
    
    try:
        logger.info('Wiping old data in SalesTerritoryHistory...')
        conn.execute('DELETE FROM SalesTerritoryHistory;')
        conn.commit()
        logger.success('Wiped old data in SalesTerritoryHistory.')
    except Exception as e:
        logger.error(f'Error wiping old data in SalesTerritoryHistory. Error: {e}')
        return
    
    try:
        logger.info('Connecting to the SalesTerritoryHistory data sources...')
        aw_sales_territory_history = pd.read_sql('SELECT * FROM "Sales.SalesTerritoryHistory"', adventureworks_sqlite_conn)
        logger.success('Connected to the SalesTerritoryHistory data sources.')
    except Exception as e:
        logger.error(f'Error connecting to the SalesTerritoryHistory data sources. Error: {e}')
        return

    surrogate_key = 1

    try:
        logger.info('Transferring the SalesTerritoryHistory data from AdventureWorks to SalesTerritoryHistory...')

        for index, row in aw_sales_territory_history.iterrows():
            
            query = """
                INSERT INTO SalesTerritoryHistory (
                    TerritoryID,
                    BusinessEntityID,
                    StartDate,
                    EndDate,
                    ModifiedDate,
                    SK
                ) VALUES (?, ?, ?, ?, ?, ?);
            """

            params = (
                row['TerritoryID'],
                row['BusinessEntityID'],
                row['StartDate'],
                row['EndDate'],
                row['ModifiedDate'],
                surrogate_key
            )

            conn.execute(query, params)
            conn.commit()

            surrogate_key += 1

        logger.success('Transferred the SalesTerritoryHistory data from AdventureWorks to SalesTerritoryHistory.')

    except Exception as e:
        logger.error(f'Error transferring the SalesTerritoryHistory data from AdventureWorks to SalesTerritoryHistory. Error: {e}')
        return

def create_empty_salesterritoryhistory_dataframe():
    dtypes = {
        'TerritoryID': np.int32,
        'BusinessEntityID': np.int32,
        'StartDate': 'datetime64[ns]',
        'EndDate': 'datetime64[ns]',
        'ModifiedDate': 'datetime64[ns]',
        'SK': np.int32
    }

    df = pd.DataFrame(columns=dtypes.keys()).astype(dtypes)
    return df

test_database = sqlite3.connect('../data/processed/test.sqlite')
create_empty_salesterritoryhistory_dataframe().to_sql('SalesTerritoryHistory', test_database, index=False, if_exists='replace')

transfer_data_to_salesterritoryhistory(test_database)

2024-05-21 18:44:41.813 | INFO     | __main__:transfer_data_to_salesterritoryhistory:4 - Wiping old data in SalesTerritoryHistory...
2024-05-21 18:44:41.912 | SUCCESS  | __main__:transfer_data_to_salesterritoryhistory:7 - Wiped old data in SalesTerritoryHistory.
2024-05-21 18:44:41.913 | INFO     | __main__:transfer_data_to_salesterritoryhistory:13 - Connecting to the SalesTerritoryHistory data sources...
2024-05-21 18:44:41.915 | SUCCESS  | __main__:transfer_data_to_salesterritoryhistory:15 - Connected to the SalesTerritoryHistory data sources.
2024-05-21 18:44:41.916 | INFO     | __main__:transfer_data_to_salesterritoryhistory:23 - Transferring the SalesTerritoryHistory data from AdventureWorks to SalesTerritoryHistory...
2024-05-21 18:44:43.495 | SUCCESS  | __main__:transfer_data_to_salesterritoryhistory:52 - Transferred the SalesTerritoryHistory data from AdventureWorks to SalesTerritoryHistory.


### Territory

In [8]:
def transfer_data_to_territory(conn):
    
    try:
        logger.info('Wiping old data in Territory...')
        conn.execute('DELETE FROM Territory;')
        conn.commit()
        logger.success('Wiped old data in Territory.')
    except Exception as e:
        logger.error(f'Error wiping old data in Territory. Error: {e}')
        return
    
    try:
        logger.info('Connecting to the Territory data sources...')
        aw_sales_territory = pd.read_sql('SELECT * FROM "Sales.SalesTerritory"', adventureworks_sqlite_conn)
        northwind_territory = pd.read_sql('SELECT * FROM Territories', northwind_sqlite_conn)
        northwind_region = pd.read_sql('SELECT * FROM Region', northwind_sqlite_conn)
        logger.success('Connected to the Territory data sources.')
    except Exception as e:
        logger.error(f'Error connecting to the Territory data sources. Error: {e}')
        return

    surrogate_key = 1

    try:
        logger.info('Transferring the Territory data from AdventureWorks to Territory...')

        for index, row in aw_sales_territory.iterrows():
            
            query = """
                INSERT INTO Territory (
                    TerritoryID,
                    CountryRegionCode,
                    Name,
                    GeographicRegion,
                    SalesYTD,
                    SalesLastYear,
                    CostYTD,
                    CostLastYear,
                    ModifiedDate,
                    SK
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
            """

            params = (
                row['TerritoryID'],
                row['CountryRegionCode'],
                row['Name'],
                row['Group'],
                row['SalesYTD'],
                row['SalesLastYear'],
                row['CostYTD'],
                row['CostLastYear'],
                row['ModifiedDate'],
                surrogate_key
            )

            conn.execute(query, params)
            conn.commit()

            surrogate_key += 1

        logger.success('Transferred the Territory data from AdventureWorks to Territory.')

    except Exception as e:
        logger.error(f'Error transferring the Territory data from AdventureWorks to Territory. Error: {e}')
        return

    try:

        logger.info('Transferring the Territory data from Northwind to Territory...')

        for index, row in northwind_territory.iterrows():

            region = northwind_region[northwind_region['RegionID'] == row['RegionID']]['RegionDescription'].values[0]
            
            query = """
                INSERT INTO Territory (
                    TerritoryID,
                    CountryRegionCode,
                    Name,
                    TerritoryDescription,
                    SK
                ) VALUES (?, ?, ?, ?, ?);
            """

            params = (
                row['TerritoryID'],
                row['RegionID'],
                region,
                row['TerritoryDescription'],
                surrogate_key
            )

            conn.execute(query, params)
            conn.commit()

            surrogate_key += 1
        
        logger.success('Transferred the Territory data from Northwind to Territory.')
    
    except Exception as e:
        logger.error(f'Error transferring the Territory data from Northwind to Territory. Error: {e}')
        return

def create_empty_territory_dataframe():
    dtypes = {
        'TerritoryID': np.int32,
        'CountryRegionCode': str,
        'Name': str,
        'GeographicRegion': str,
        'TerritoryDescription': str,
        'SalesYTD': np.float64,
        'SalesLastYear': np.float64,
        'CostYTD': np.float64,
        'CostLastYear': np.float64,
        'ModifiedDate': 'datetime64[ns]',
        'SK': np.int32
    }

    df = pd.DataFrame(columns=dtypes.keys()).astype(dtypes)
    return df

test_database = sqlite3.connect('../data/processed/test.sqlite')
create_empty_territory_dataframe().to_sql('Territory', test_database, index=False, if_exists='replace')

transfer_data_to_territory(test_database)

2024-05-21 19:38:56.509 | INFO     | __main__:transfer_data_to_territory:4 - Wiping old data in Territory...
2024-05-21 19:38:56.583 | SUCCESS  | __main__:transfer_data_to_territory:7 - Wiped old data in Territory.
2024-05-21 19:38:56.584 | INFO     | __main__:transfer_data_to_territory:13 - Connecting to the Territory data sources...
2024-05-21 19:38:56.586 | SUCCESS  | __main__:transfer_data_to_territory:17 - Connected to the Territory data sources.
2024-05-21 19:38:56.587 | INFO     | __main__:transfer_data_to_territory:25 - Transferring the Territory data from AdventureWorks to Territory...
2024-05-21 19:38:57.458 | SUCCESS  | __main__:transfer_data_to_territory:62 - Transferred the Territory data from AdventureWorks to Territory.
2024-05-21 19:38:57.458 | INFO     | __main__:transfer_data_to_territory:70 - Transferring the Territory data from Northwind to Territory...
2024-05-21 19:39:01.993 | SUCCESS  | __main__:transfer_data_to_territory:99 - Transferred the Territory data from 

### CreditCard

In [10]:
def transfer_data_to_creditcard(conn):

    try:
        logger.info('Wiping old data in CreditCard...')
        conn.execute('DELETE FROM CreditCard;')
        conn.commit()
        logger.success('Wiped old data in CreditCard.')
    except Exception as e:
        logger.error(f'Error wiping old data in CreditCard. Error: {e}')
        return
    
    try:
        logger.info('Connecting to the CreditCard data sources...')
        aw_credit_card = pd.read_sql('SELECT * FROM "Sales.CreditCard"', adventureworks_sqlite_conn)
        aw_person_credit_card = pd.read_sql('SELECT * FROM "Sales.PersonCreditCard"', adventureworks_sqlite_conn)
        logger.success('Connected to the CreditCard data sources.')
    except Exception as e:
        logger.error(f'Error connecting to the CreditCard data sources. Error: {e}')
        return

    try:
        logger.info('Transferring the CreditCard data from AdventureWorks to CreditCard...')

        for index, row in aw_credit_card.iterrows():

            business_entity_id = aw_person_credit_card[aw_person_credit_card['CreditCardID'] == row['CreditCardID']]['BusinessEntityID'].values[0]
            
            query = """
                INSERT INTO CreditCard (
                    CreditCardID,
                    BusinessEntityID,
                    CardNumber,
                    CardType,
                    ExpMonth,
                    ExpYear,
                    ModifiedDate
                ) VALUES (?, ?, ?, ?, ?, ?, ?);
            """

            params = (
                row['CreditCardID'],
                int(business_entity_id),
                row['CardNumber'],
                row['CardType'],
                row['ExpMonth'],
                row['ExpYear'],
                row['ModifiedDate']
            )

            conn.execute(query, params)
            conn.commit()

        logger.success('Transferred the CreditCard data from AdventureWorks to CreditCard.')

    except Exception as e:
        logger.error(f'Error transferring the CreditCard data from AdventureWorks to CreditCard. Error: {e}')
        return

def create_empty_creditcard_dataframe():
    dtypes = {
        'CreditCardID': np.int32,
        'BusinessEntityID': np.int32,
        'CardNumber': np.int32,
        'CardType': str,
        'ExpMonth': str,
        'ExpYear': str,
        'ModifiedDate': 'datetime64[ns]'
    }

    df = pd.DataFrame(columns=dtypes.keys()).astype(dtypes)
    return df

test_database = sqlite3.connect('../data/processed/test.sqlite')
create_empty_creditcard_dataframe().to_sql('CreditCard', test_database, index=False, if_exists='replace')

transfer_data_to_creditcard(test_database)

2024-05-21 20:16:15.690 | INFO     | __main__:transfer_data_to_creditcard:4 - Wiping old data in CreditCard...
2024-05-21 20:16:15.764 | SUCCESS  | __main__:transfer_data_to_creditcard:7 - Wiped old data in CreditCard.
2024-05-21 20:16:15.764 | INFO     | __main__:transfer_data_to_creditcard:13 - Connecting to the CreditCard data sources...
2024-05-21 20:16:15.833 | SUCCESS  | __main__:transfer_data_to_creditcard:16 - Connected to the CreditCard data sources.
2024-05-21 20:16:15.834 | INFO     | __main__:transfer_data_to_creditcard:22 - Transferring the CreditCard data from AdventureWorks to CreditCard...
2024-05-21 20:52:43.564 | SUCCESS  | __main__:transfer_data_to_creditcard:53 - Transferred the CreditCard data from AdventureWorks to CreditCard.


### CurrencyRate

In [24]:
def transfer_data_to_currencyrate(conn):

    try:
        logger.info('Wiping old data in CurrencyRate...')
        conn.execute('DELETE FROM CurrencyRate;')
        conn.commit()
        logger.success('Wiped old data in CurrencyRate.')
    except Exception as e:
        logger.error(f'Error wiping old data in CurrencyRate. Error: {e}')
        return
    
    try:
        logger.info('Connecting to the CurrencyRate data sources...')
        aw_currency_rate = pd.read_sql('SELECT * FROM "Sales.CurrencyRate"', adventureworks_sqlite_conn)
        aw_currency = pd.read_sql('SELECT * FROM "Sales.Currency"', adventureworks_sqlite_conn)
        logger.success('Connected to the CurrencyRate data sources.')
    except Exception as e:
        logger.error(f'Error connecting to the CurrencyRate data sources. Error: {e}')
        return

    try:
        logger.info('Transferring the CurrencyRate data from AdventureWorks to CurrencyRate...')

        aw_currency_rate = pd.read_sql('SELECT * FROM "Sales.CurrencyRate"', adventureworks_sqlite_conn)
        aw_currency = pd.read_sql('SELECT * FROM "Sales.Currency"', adventureworks_sqlite_conn)

        aw_currency.rename(columns={'Name': 'CurrencyName'}, inplace=True)

        currency_rate = aw_currency_rate.merge(aw_currency, left_on='FromCurrencyCode', right_on='CurrencyCode', how='left')
        currency_rate.rename(columns={'CurrencyName': 'FromName'}, inplace=True)
        currency_rate.drop(columns='CurrencyCode', inplace=True)

        currency_rate = currency_rate.merge(aw_currency, left_on='ToCurrencyCode', right_on='CurrencyCode', how='left')
        currency_rate.rename(columns={'CurrencyName': 'ToName'}, inplace=True)
        currency_rate.drop(columns='CurrencyCode', inplace=True)

        currency_rate = currency_rate[[
            'CurrencyRateID',
            'CurrencyRateDate',
            'FromCurrencyCode',
            'FromName',
            'ToCurrencyCode',
            'ToName',
            'AverageRate',
            'EndOfDayRate',
            'ModifiedDate'
        ]]

        currency_rate.to_sql('CurrencyRate', conn, index=False, if_exists='replace')

        logger.success('Transferred the CurrencyRate data from AdventureWorks to CurrencyRate.')
    except Exception as e:
        logger.error(f'Error transferring the CurrencyRate data from AdventureWorks to CurrencyRate. Error: {e}')
        return
    
def create_empty_currencyrate_dataframe():
    dtypes = {
        'CurrencyRateID': np.int32,
        'AverageRate': np.float64,
        'CurrencyRateDate': 'datetime64[ns]',
        'EndOfDayRate': 'datetime64[ns]',
        'FromCurrencyCode': str,
        'FromName': str,
        'ToCurrencyCode': str,
        'ToName': str,
        'ModifiedDate': 'datetime64[ns]'
    }

    df = pd.DataFrame(columns=dtypes.keys()).astype(dtypes)
    return df

test_database = sqlite3.connect('../data/processed/test.sqlite')
create_empty_currencyrate_dataframe().to_sql('CurrencyRate', test_database, index=False, if_exists='replace')

transfer_data_to_currencyrate(test_database)

2024-05-22 20:13:10.922 | INFO     | __main__:transfer_data_to_currencyrate:4 - Wiping old data in CurrencyRate...
2024-05-22 20:13:11.029 | SUCCESS  | __main__:transfer_data_to_currencyrate:7 - Wiped old data in CurrencyRate.
2024-05-22 20:13:11.030 | INFO     | __main__:transfer_data_to_currencyrate:13 - Connecting to the CurrencyRate data sources...
2024-05-22 20:13:11.073 | SUCCESS  | __main__:transfer_data_to_currencyrate:16 - Connected to the CurrencyRate data sources.
2024-05-22 20:13:11.074 | INFO     | __main__:transfer_data_to_currencyrate:22 - Transferring the CurrencyRate data from AdventureWorks to CurrencyRate...
2024-05-22 20:13:11.645 | SUCCESS  | __main__:transfer_data_to_currencyrate:51 - Transferred the CurrencyRate data from AdventureWorks to CurrencyRate.


### CountryRegionCurrency

In [7]:
def transfer_data_to_currency(conn):
    
    try:
        logger.info('Wiping old data in Currency...')
        conn.execute('DELETE FROM Currency;')
        conn.commit()
        logger.success('Wiped old data in Currency.')
    except Exception as e:
        logger.error(f'Error wiping old data in Currency. Error: {e}')
        return
    
    try:
        logger.info('Connecting to the Currency and Country Region data sources...')
        currency = pd.read_sql('SELECT * FROM "Sales.Currency"', adventureworks_sqlite_conn).rename(columns={'Name': 'CurrencyName'})
        logger.success('Connected to the Currency and Country Region data sources.')
    except Exception as e:
        logger.error(f'Error connecting to the Currency and Country Region data sources. Error: {e}')
        return

    try:
        logger.info('Transferring the Currency and Country Region data from AdventureWorks to Currency...')

        countryregioncurrency = pd.read_sql('SELECT * FROM "Sales.CountryRegionCurrency"', adventureworks_sqlite_conn).drop(columns='ModifiedDate')
        countryregion = pd.read_sql('SELECT * FROM "Person.CountryRegion"', adventureworks_sqlite_conn).drop(columns='ModifiedDate').rename(columns={'Name': 'CountryRegionName'})

        currency = pd.merge(currency, countryregioncurrency, on='CurrencyCode', how='left')
        currency = pd.merge(currency, countryregion, on='CountryRegionCode', how='left')
        currency = currency[['CurrencyCode', 'CurrencyName', 'CountryRegionCode', 'CountryRegionName', 'ModifiedDate']]

        currency.to_sql('Currency', conn, index=False, if_exists='replace')

        logger.success('Transferred the Currency and Country Region data from AdventureWorks to Currency.')
    except Exception as e:
        logger.error(f'Error transferring the Currency and Country Region data from AdventureWorks to Currency. Error: {e}')
        return
    
def create_empty_currency_dataframe():
    dtypes = {
        'CurrencyCode': str,
        'CurrencyName': str,
        'CountryRegionCode': str,
        'CountryRegionName': str,
        'ModifiedDate': 'datetime64[ns]'
    }

    df = pd.DataFrame(columns=dtypes.keys()).astype(dtypes)
    return df

test_database = sqlite3.connect('../data/processed/test.sqlite')
create_empty_currency_dataframe().to_sql('Currency', test_database, index=False, if_exists='replace')

transfer_data_to_currency(test_database)

2024-05-23 10:33:52.961 | INFO     | __main__:transfer_data_to_currency:4 - Wiping old data in Currency...
2024-05-23 10:33:52.969 | SUCCESS  | __main__:transfer_data_to_currency:7 - Wiped old data in Currency.
2024-05-23 10:33:52.969 | INFO     | __main__:transfer_data_to_currency:13 - Connecting to the Currency and Country Region data sources...
2024-05-23 10:33:52.979 | SUCCESS  | __main__:transfer_data_to_currency:15 - Connected to the Currency and Country Region data sources.
2024-05-23 10:33:52.979 | INFO     | __main__:transfer_data_to_currency:21 - Transferring the Currency and Country Region data from AdventureWorks to Currency...
2024-05-23 10:33:53.029 | SUCCESS  | __main__:transfer_data_to_currency:32 - Transferred the Currency and Country Region data from AdventureWorks to Currency.


### SpecialOffer

In [8]:
def transfer_data_to_specialoffer(conn):

    try:
        logger.info('Wiping old data in SpecialOffer...')
        conn.execute('DELETE FROM SpecialOffer;')
        conn.commit()
        logger.success('Wiped old data in SpecialOffer.')
    except Exception as e:
        logger.error(f'Error wiping old data in SpecialOffer. Error: {e}')
        return
    
    try:
        logger.info('Connecting to the SpecialOffer data sources...')
        aw_special_offer = pd.read_sql('SELECT * FROM "Sales.SpecialOffer"', adventureworks_sqlite_conn)
        logger.success('Connected to the SpecialOffer data sources.')
    except Exception as e:
        logger.error(f'Error connecting to the SpecialOffer data sources. Error: {e}')
        return

    try:
        logger.info('Transferring the SpecialOffer data from AdventureWorks to SpecialOffer...')

        aw_special_offer.to_sql('SpecialOffer', conn, index=False, if_exists='replace')

        logger.success('Transferred the SpecialOffer data from AdventureWorks to SpecialOffer.')
    except Exception as e:
        logger.error(f'Error transferring the SpecialOffer data from AdventureWorks to SpecialOffer. Error: {e}')
        return

def create_empty_special_offer_dataframe():
    dtypes = {
        'SpecialOfferID': 'int64',
        'Description': str,
        'DiscountPct': 'float64',
        'Type': str,
        'Category': str,
        'StartDate': 'datetime64[ns]',
        'EndDate': 'datetime64[ns]',
        'MinQty': 'int64',
        'MaxQty': 'int64',
        'ModifiedDate': 'datetime64[ns]'
    }

    df = pd.DataFrame(columns=dtypes.keys()).astype(dtypes)
    return df

test_database = sqlite3.connect('../data/processed/test.sqlite')
create_empty_special_offer_dataframe().to_sql('SpecialOffer', test_database, index=False, if_exists='replace')

transfer_data_to_specialoffer(test_database)

2024-05-23 10:47:40.092 | INFO     | __main__:transfer_data_to_specialoffer:4 - Wiping old data in SpecialOffer...
2024-05-23 10:47:40.101 | SUCCESS  | __main__:transfer_data_to_specialoffer:7 - Wiped old data in SpecialOffer.
2024-05-23 10:47:40.106 | INFO     | __main__:transfer_data_to_specialoffer:13 - Connecting to the SpecialOffer data sources...
2024-05-23 10:47:40.116 | SUCCESS  | __main__:transfer_data_to_specialoffer:15 - Connected to the SpecialOffer data sources.
2024-05-23 10:47:40.116 | INFO     | __main__:transfer_data_to_specialoffer:21 - Transferring the SpecialOffer data from AdventureWorks to SpecialOffer...
2024-05-23 10:47:40.133 | SUCCESS  | __main__:transfer_data_to_specialoffer:25 - Transferred the SpecialOffer data from AdventureWorks to SpecialOffer.


### Connecties sluiten

In [19]:
northwind_sqlite_conn.close()
adventureworks_sqlite_conn.close()
northwind_ssms_conn.close()
adventureworks_ssms_conn.close()
aenc_access_conn.close()
aenc_sqlite_conn.close()